## Part 1: Preprocessing

In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [3]:
# Determine the number of unique values in each column.
attrition_df.nunique()

,0
Age,43
Attrition,2
BusinessTravel,3
Department,3
DistanceFromHome,29
Education,5
EducationField,6
EnvironmentSatisfaction,4
HourlyRate,71
JobInvolvement,4


In [4]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]


In [5]:
# Create a list of at least 10 column names to use as X data
X_columns = ['Education', 'Age', 'DistanceFromHome', 'JobSatisfaction', 'OverTime',
             'StockOptionLevel', 'WorkLifeBalance', 'YearsAtCompany',
             'YearsSinceLastPromotion', 'NumCompaniesWorked']


# Create X_df using your selected columns
X_df = attrition_df[X_columns]


# Show the data types for X_df
print(X_df.dtypes)


Education                   int64
Age                         int64
DistanceFromHome            int64
JobSatisfaction             int64
OverTime                   object
StockOptionLevel            int64
WorkLifeBalance             int64
YearsAtCompany              int64
YearsSinceLastPromotion     int64
NumCompaniesWorked          int64
dtype: object


In [6]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=42)


In [15]:
# Check the original distribution in the full dataset
print(attrition_df['OverTime'].value_counts())
print(f"Name: OverTime, dtype: {attrition_df['OverTime'].dtype}")

OverTime
No     1054
Yes     416
Name: count, dtype: int64
Name: OverTime, dtype: object


In [17]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary

# Print the value counts without the index name
print(X_train['OverTime'].value_counts().to_string())

# Print the dtype on a new line
print(f"Name: OverTime, dtype: {X_train['OverTime'].dtype}")

OverTime
No    1102
Name: OverTime, dtype: object


In [18]:
# Create a StandardScaler
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
# We'll exclude 'OverTime' as it's categorical
numeric_columns = X_train.select_dtypes(include=['int64', 'float64']).columns
X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()

X_train_scaled[numeric_columns] = scaler.fit_transform(X_train[numeric_columns])

# Scale the testing data
X_test_scaled[numeric_columns] = scaler.transform(X_test[numeric_columns])


In [24]:
# Create a OneHotEncoder for the Department column
dept_encoder = OneHotEncoder(sparse_output=False, drop='first')

# Fit the encoder to the training data
dept_encoded_train = dept_encoder.fit_transform(y_train[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data
dept_encoded_test = dept_encoder.transform(y_test[['Department']])

print(dept_encoded_train[:5])  # Print first 5 rows for brevity

[[1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]]


In [25]:
# Create a OneHotEncoder for the Attrition column
attrition_encoder = OneHotEncoder(sparse_output=False, drop='first')

# Fit the encoder to the training data
attrition_encoded_train = attrition_encoder.fit_transform(y_train[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data
attrition_encoded_test = attrition_encoder.transform(y_test[['Attrition']])

print(attrition_encoded_train[:5])  # Print first 5 rows for brevity


[[0.]
 [0.]
 [0.]
 [0.]
 [0.]]


## Create, Compile, and Train the Model

In [26]:
# Find the number of columns in the X training data
n_features = X_train.shape[1]

# Create the input layer
input_layer = layers.Input(shape=(n_features,))

# Create at least two shared layers
shared_layer1 = layers.Dense(64, activation='relu')(input_layer)
shared_layer2 = layers.Dense(32, activation='relu')(shared_layer1)

In [27]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
dept_hidden = layers.Dense(16, activation='relu')(shared_layer2)

# Create the output layer
dept_output = layers.Dense(2, activation='softmax', name='department_output')(dept_hidden)


In [28]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_hidden = layers.Dense(16, activation='relu')(shared_layer2)

# Create the output layer
attrition_output = layers.Dense(1, activation='sigmoid', name='attrition_output')(attrition_hidden)

In [29]:
# Create the model
model = Model(inputs=input_layer, outputs=[dept_output, attrition_output])

# Compile the model
model.compile(optimizer='adam',
              loss={'department_output': 'categorical_crossentropy',
                    'attrition_output': 'binary_crossentropy'},
              metrics={'department_output': 'accuracy',
                       'attrition_output': 'accuracy'})

# Summarize the model
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 10)             │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 64)             │            704 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 32)             │          2,080 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 16)             │            528 │ dense_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_3 (Dense)           │ (None, 16)             │            528 │ dense_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ department_output (Dense) │ (None, 2)              │             34 │ dense_2[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attrition_output (Dense)  │ (None, 1)              │             17 │ dense_3[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 3,891 (15.20 KB)

 Trainable params: 3,891 (15.20 KB)

 Non-trainable params: 0 (0.00 B)

In [35]:
# Train the model
history = model.fit(X_train_scaled,
                    [dept_encoded_train, attrition_encoded_train],
                    epochs=100,
                    batch_size=32,
                    validation_split=0.2,
                    verbose=0)

In [45]:
# Evaluate the model with the testing data
evaluation_results = model.evaluate(X_test_scaled, [dept_encoded_test, attrition_encoded_test], verbose=1)

print(evaluation_results)

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - attrition_output_accuracy: 0.8492 - department_output_accuracy: 0.6916 - loss: nan 
[nan, 0.8695651888847351, 0.7010869383811951]


In [47]:
# Print the accuracies
print(f"Department accuracy: {evaluation_results[1]:.2%}")
print(f"Attrition accuracy: {evaluation_results[2]:.2%}")

Department accuracy: 86.96%
Attrition accuracy: 70.11%


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1.  Accuracy may not be the best metric for this data, especially for the Attrition prediction:
   - For Department prediction, accuracy could be appropriate if the classes are balanced.
   - For Attrition prediction, accuracy can be misleading if the classes are imbalanced (which is common in employee attrition data). Metrics like precision, recall, F1-score, or AUC-ROC might be more informative, as they account for class imbalance and provide a more comprehensive view of the model's performance.

2. The activation functions chosen for the output layers are:
   - For Department output: Softmax activation, because it's a multi-class classification problem. Softmax ensures the outputs sum to 1, which can be interpreted as probabilities for each department.
   - For Attrition output: Sigmoid activation, because it's a binary classification problem. Sigmoid squashes the output between 0 and 1, which can be interpreted as the probability of attrition.

3.  Ways this model might be improved include:
   - Feature engineering: Create new features or transform existing ones to capture more relevant information.
   - Hyperparameter tuning: Use techniques like grid search or random search to optimize model parameters.
   - Address class imbalance: Use techniques like oversampling, undersampling, or SMOTE if the classes are imbalanced, especially for the Attrition prediction.
   - Try different model architectures: Experiment with different layer sizes, types (e.g., adding dropout layers), or even different model types like Random Forests or Gradient Boosting.
   - Collect more data if possible, especially for the lower-performing prediction task (Attrition in this case).
   - Use regularization techniques to prevent overfitting if that's an issue.
   - Ensemble methods: Combine multiple models to improve overall performance.